In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

# ===== CONFIGURATION PARAMETERS =====
# Ruta al CSV de referencia
input_csv = '../Datasets/ml-small.csv'
# Ruta de salida para el CSV sintético
output_csv = '../Datasets/sintetic.csv'
# Número de clusters para K-Means
K = 100
# Número de usuarios sintéticos a generar (None = mismo número que en el dataset de referencia)
U_total = None  # o un entero, por ejemplo 1000
# Factor de densidad para el número de ítems por usuario
density_factor = 1 # o un float, por ejemplo 0.5


def learn_distributions(df: pd.DataFrame, K: int):
    """
    Aprende las distribuciones empíricas a partir de un subset de datos para un valor de rating.

    Devuelve un diccionario con:
    - PC: array de probabilidades de usuario por cluster
    - PU: dict cluster -> {número_de_ratings_por_usuario: probabilidad}
    - PI: dict cluster -> {índice_item: conteo}
    - users: lista de userIds en la referencia
    - items: lista de itemIds en la referencia
    """
    users = df['userId'].unique()
    items = df['itemId'].unique()
    user_idx = {u: i for i, u in enumerate(users)}
    item_idx = {it: i for i, it in enumerate(items)}

    # Matriz binaria usuario-item
    mat = np.zeros((len(users), len(items)), dtype=int)
    for _, row in df.iterrows():
        mat[user_idx[row.userId], item_idx[row.itemId]] = 1

    # Clustering con K-Means
    kmeans = KMeans(n_clusters=K, random_state=0)
    labels = kmeans.fit_predict(mat)

    # Distribución PC: usuarios por cluster
    PC = np.bincount(labels, minlength=K) / len(users)

    # Distribución PU: ratings por usuario en cada cluster
    PU = {}
    for k in range(K):
        idx = np.where(labels == k)[0]
        counts = mat[idx].sum(axis=1)
        vals, cnts = np.unique(counts, return_counts=True)
        PU[k] = dict(zip(vals.tolist(), (cnts / len(idx)).tolist()))

    # Distribución PI: ratings por ítem en cada cluster (peso para muestreo)
    PI = {}
    for k in range(K):
        idx = np.where(labels == k)[0]
        counts = mat[idx].sum(axis=0)
        PI[k] = dict(enumerate(counts.tolist()))

    return {
        'PC': PC,
        'PU': PU,
        'PI': PI,
        'users': users,
        'items': items
    }


def generate_synthetic(dist: dict, U: int, rating_value: float):
    """
    Genera U usuarios sintéticos para un valor de rating usando las distribuciones aprendidas.
    Devuelve un DataFrame con columnas userId, itemId, rating.
    """
    PC = dist['PC']
    PU = dist['PU']
    PI = dist['PI']
    items = dist['items']
    K = len(PC)

    rows = []
    for user_id in range(U):
        # Asignar cluster según PC
        k = np.random.choice(np.arange(K), p=PC)
        # Muestrear número de ítems para el usuario
        counts = list(PU[k].keys())
        probs = list(PU[k].values())
        I = np.random.choice(counts, p=probs) * density_factor
        # I tipo int
        I = int(I)
        
        # Muestrear ítems sin reemplazo, ponderado por popularidad en el cluster
        weights = np.array([PI[k].get(i, 0) for i in range(len(items))], dtype=float)
        if weights.sum() <= 0:
            weights = np.ones(len(items))
        weights /= weights.sum()

        nonzero_weights = np.count_nonzero(weights)
        I = min(I, nonzero_weights)
        
        chosen = np.random.choice(
            np.arange(len(items)), size=min(I, len(items)), replace=False, p=weights
        )
        for idx in chosen:
            rows.append((user_id, items[idx], rating_value))

    return pd.DataFrame(rows, columns=['userId', 'itemId', 'rating'])


# ===== EJECUCIÓN PRINCIPAL =====
# Carga del dataset de referencia
df = pd.read_csv(input_csv)

# Determinar número de usuarios sintéticos si no se especifica
if U_total is None:
    U_total = df['userId'].nunique()

# Generar datasets sintéticos por cada valor de rating
synth_parts = []
for r in np.sort(df['rating'].unique()):
    df_r = df[df['rating'] == r]
    dist = learn_distributions(df_r, K)
    synth_r = generate_synthetic(dist, U_total, r)
    synth_parts.append(synth_r)

# Concatenar y guardar
synth_all = pd.concat(synth_parts, ignore_index=True)
# drop duplicates
synth_all = synth_all.drop_duplicates(subset=['userId', 'itemId'])
synth_all.to_csv(output_csv, index=False)

print(f"Dataset sintético generado en '{output_csv}' con {U_total} usuarios.")


Dataset sintético generado en '../Datasets/sintetic.csv' con 610 usuarios.
